In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from numpy.random import seed
seed(1)
tf.random.set_seed(1)
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Input
from matplotlib import pyplot as plt
%matplotlib inline
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from PIL import Image
from numpy import asarray

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
daniela_path = 'drive/MyDrive/Class/CS209b/CS209b_Final_Project'

path = daniela_path

In [ ]:
# Load predictor datasets from census and polls
historical_polls = pd.read_csv(path + "house_polls_historical.csv")
house_polls = pd.read_csv(path + "house_polls.csv")
occ18 = pd.read_csv(path + "occupation_data_18.csv")
occ22 = pd.read_csv(path + "occupation_data_22.csv")
edu17 = pd.read_csv(path + "census/education_2017.csv").drop(0, axis=0)
edu21 = pd.read_csv(path + "census/education_2021.csv").drop(0, axis=0)
hhinc15 = pd.read_csv(path + "census/hhinc_2015.csv").drop(0, axis=0)
hhinc17 = pd.read_csv(path + "census/HHINC_2017.csv").drop(0, axis=0)
hhinc19 = pd.read_csv(path + "census/hhinc_2019.csv").drop(0, axis=0)
hhinc21 = pd.read_csv(path + "census/HHINC_2021.csv").drop(0, axis=0)
age_sex17 = pd.read_csv(path + "census/census_sex_age_2017.csv")
age_sex21 = pd.read_csv(path + "census/census_sex_age_2021.csv")
race17 = pd.read_csv(path + "census/race2017.csv")
race21 = pd.read_csv(path + "census/race2021.csv")

In [ ]:
# Remove unnecessary columns
election2018 = election2018.drop(['GE RUNOFF ELECTION VOTES (MS Senate)',
       'GE RUNOFF ELECTION % (MS Senate)', 'COMBINED GE PARTY TOTALS (CT, NY)',
       'COMBINED % (CT, NY)', 'FOOTNOTES', 'Unnamed: 23', 'Unnamed: 24'], axis=1)
# Rename columns with stripped column names
columns = []
for column in election2018.columns:
    columns.append(column.strip())
election2018.columns = columns

# Cleaning candidates' affiliated party to only Republican/Democrat
election2018['PARTY'] = election2018['PARTY'].apply(lambda x: str(x).strip())
election2018.loc[election2018['PARTY'] == 'D/IP', 'PARTY'] = "D"
election2018.loc[election2018['PARTY'] == 'R/TRP', 'PARTY'] = "R"
election2018 = election2018[(election2018['PARTY'] == "R") | (election2018['PARTY'] == "D")]

# Subsetting to only include winners
election2018 = election2018[election2018['GE WINNER INDICATOR'] == "W"]

In [ ]:
election18 = pd.read_csv(path + "/election2018_full_v2.csv")
#election22 = pd.read_csv(path + "/election2022_full.csv")

In [ ]:
# Google Street View Data: 
# https://developers.google.com/maps/documentation/javascript/reference/street-view#StreetViewCoverageLayer
# http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/zipped%20images/ 

In [ ]:
# Initialise an image generator object
generator = ImageDataGenerator(rescale=1./255)

# Initialising number of data images
num_data = len(os.listdir(path + '/data/part10/'))

print("NUM_DATA: " + str(num_data))

# Read the image data from the directory using the generator object
img_generator = generator.flow_from_directory(directory=path + "/data/part10/", color_mode='rgb', seed=1,
                                    batch_size=16, target_size=(150, 150), class_mode='binary')

# Print the target size i.e. the total dataset size
TARGET_SIZE = img_generator.target_size
print(f'Generator produces images of size {TARGET_SIZE} (with 3 color channels)')

# Print the batch size
BATCH_SIZE = img_generator.batch_size
print(f'Images are generated in batches of size {BATCH_SIZE}')

In [ ]:
# Fixing the random seed
seed(1)
tf.random.set_seed(1)

# Creating a feed-forward Neural Network
FFNN = Sequential()

# Specify a layer that takes the input with input shape
# the same as the size of the image defined during image generation
# Remember to take into account that the image has 3 channels
FFNN.add(Input(shape=(150, 150, 3)))#tf.keras.layers.Conv2D(100, 2, 16, input_shape=(150, 150, 3)))

# Add a flatten layer to enable FFNN to process images
FFNN.add(Flatten())

# Specify a list of the number of nodes for each dense layer
ffnn_filters = [60,40,20]

# Add dense layers for the number of nodes in ffnn_filters with ReLU activation
for n_nodes in ffnn_filters:
    FFNN.add(Dense(n_nodes, activation='relu'))


# Add the final dense layer with 1 output node to differentiate 
# between the two classes and sigmoid activation
FFNN.add(Dense(1, activation='sigmoid'))

# Compile the model with accuracy as the metric and adam optimizer
FFNN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
